## [DO NOT CHANGE THIS CELL] Import Dependencies
#### Do not use additional Packages! USE ONLY BELOW IMPORTED ONES

In [ ]:
import os
import csv
import random
import numpy as np

import torch
import torch.nn as nn
from tqdm.notebook import tqdm

## [DO NOT CHANGE THIS CELL] Define file path (We use default data from colab)

In [ ]:
train_csv_fp = './sample_data/california_housing_train.csv'
test_csv_fp = './sample_data/california_housing_test.csv'

## [DO NOT CHANGE THIS CELL] Define function for read CSV files

In [ ]:
def read_csv(csv_fp):
    data = dict()
    all_names = []
    with open(csv_fp) as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i == 0:
                for name in row:
                    data[name] = list()
                    all_names.append(name)
            else:
                for ii, val in enumerate(row):
                    data[all_names[ii]].append(float(val))
    
    return data, all_names

## [DO NOT CHANGE THIS CELL] Read CSV files as a DICTIONARY

In [ ]:
train_data, names = read_csv(train_csv_fp)
test_data, _ = read_csv(test_csv_fp)
print('Attributes are: ', names)

Attributes are:  ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']


## [DO NOT CHANGE THIS CELL] Names of INPUT/OUTPUT

In [ ]:
input_name = names[:-1]
output_name = names[-1]
print('Inputs are: ', input_name)
print('Outputs are: ', output_name)

Inputs are:  ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
Outputs are:  median_house_value


## [DO NOT CHANGE THIS CELL] Normalize the data

In [ ]:
def normalize(data, mean=None, std=None):
    stat_flag = False
    if mean is None and std is None:
        mean = dict()
        std = dict()
        stat_flag = True
    
    for k, v in data.items():
        if stat_flag:
            mean[k] = np.mean(v)
            std[k] = np.std(v)
        data[k] = (np.array(v) - mean[k] ) / std[k]
    return data, mean, std

train_data, mean, std = normalize(train_data)
test_data, _, _ = normalize(test_data, mean, std)

## **[CHANGE ONLY BELOW CELL]** Code for Training

In [ ]:
def train(data, seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

    weight = np.random.rand(len(input_name), 1)
    bias = np.random.rand(1)

    ##### write your code to optimize weight and bias #####
    weight = nn.Parameter(torch.Tensor(weight), requires_grad = True)
    bias = nn.Parameter(torch.Tensor(bias), requires_grad = True)

    optimizer = torch.optim.Adam([weight, bias], lr=0.001)

    max_epoch = 50
    num_data = len(data[output_name])
    batch_size = num_data//200

    for epoch in tqdm(range(max_epoch)):
        rand_idx = np.arange(num_data)
        random.shuffle(rand_idx)

        total_loss = 0.0

        for i in range(0, num_data, batch_size):
            curr_batch_idx = rand_idx[i:min(num_data, i+batch_size)]

            curr_input = torch.Tensor([data[name][curr_batch_idx] for name in input_name])
            curr_answer = data[output_name][curr_batch_idx]
            curr_answer = torch.Tensor(curr_answer)
            
            curr_pred = torch.sum(curr_input * weight, dim=0) + bias

            loss = torch.mean((curr_pred - curr_answer) ** 2)
            
            total_loss += loss.item() / (num_data // batch_size)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # print('[EPOCH : {}] LOSS : {}'.format(epoch+1, total_loss))

    weight = weight.detach().numpy()
    bias = bias.detach().numpy()
    #####################s##################################

    # do not modify this return part.
    return weight, bias

##[DO NOT CHANGE THIS CELL] Code for testing

In [ ]:
def test(test_data, weight, bias):
    num_data = len(test_data[output_name])

    avg_loss = 0.0
    for i in range(num_data):
        curr_input = np.asarray([test_data[name][i] for name in input_name])
        curr_answer = test_data[output_name][i]

        curr_pred = (curr_input.T @ weight) + bias

        curr_loss = np.linalg.norm(curr_answer - curr_pred)

        avg_loss += curr_loss / num_data

    return avg_loss

##[DO NOT CHANGE THIS CELL] Code for testing

In [ ]:
max_iter = 50
total_loss = 0.0
for seed in tqdm(range(max_iter)):
    weight, bias = train(train_data, seed)

    test_loss = test(test_data, weight, bias)

    print('[SEED {}] TEST LOSS : {}'.format(seed, test_loss.item()))

    total_loss += test_loss / max_iter

print('TOTAL TEST LOSS: {}'.format(total_loss))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


[SEED 0] TEST LOSS : 0.43440659836523243


  0%|          | 0/50 [00:00<?, ?it/s]

...

[SEED 47] TEST LOSS : 0.43409807669790335


  0%|          | 0/50 [00:00<?, ?it/s]

[SEED 48] TEST LOSS : 0.43466786538089275


  0%|          | 0/50 [00:00<?, ?it/s]

[SEED 49] TEST LOSS : 0.4340550992092771
TOTAL TEST LOSS: 0.43439811947744933


##[DO NOT CHANGE THIS CELL] Code for testing

In [ ]:
######## DO NOT CHANGE THIS GRADING PART ########
THRESHOLD = 0.5
max_point = 40

if total_loss < THRESHOLD:
    your_point = max_point
else:
    your_point = np.exp( -0.25* (total_loss - THRESHOLD) ) * max_point

print('YOUR POINT : {}/{}'.format(your_point, max_point))
####################################################

YOUR POINT : 40/40
